# Table of Content

# Introduction

## Setup and Import

As always, the first step is to import the required libraries and data. Since we do not want to run the SQL query every time, we can simply import the csv file we created in the first notebook.

In [1]:
# Import packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import altair as alt
import numpy as np

from ipywidgets import HTML
from io import BytesIO
import base64

import warnings
warnings.simplefilter("ignore")

# Turn off the max column width so the images won't be truncated
pd.set_option('display.max_colwidth', None)
# Show all Columns
pd.set_option('display.max_columns', None)
pd. set_option('display.max_rows', None)
 
# Turning off the max column will display all the data
# if gathering into sets / array we might want to restrict to a few items
pd.set_option('display.max_seq_items', 50)
pd.set_option('display.width', 1000)


In [2]:
# Import dataframes for Stock Prices
stock_price_df = pd.read_csv('../data/train_files/stock_prices.csv', parse_dates=['Date'])
sec_df = pd.read_csv('../data/train_files/secondary_stock_prices.csv', parse_dates=['Date'])
tra_df = pd.read_csv('../data/train_files/trades.csv', parse_dates=['Date'])

stock_desc_df = pd.read_csv('../data/stock_price_spec.csv')
stock_list_desc_df = pd.read_csv('../data/stock_list_spec.csv')
stock_list = pd.read_csv('../data/stock_list.csv')

In [3]:
stock_price_df.head(5)

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


In [4]:
stock_list.head()

,SecuritiesCode,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,NewIndexSeriesSize,TradeDate,Close,IssuedShares,MarketCapitalization,Universe0
0,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,1.092828e+07,3.365911e+10,True
1,1305,20211230,Daiwa ETF-TOPIX,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,2097.0,3.634636e+09,7.621831e+12,False
2,1306,20211230,NEXT FUNDS TOPIX Exchange Traded Fund,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,2073.5,7.917718e+09,1.641739e+13,False
3,1308,20211230,Nikko Exchange Traded Index Fund TOPIX,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,2053.0,3.736943e+09,7.671945e+12,False
4,1309,20211230,NEXT FUNDS ChinaAMC SSE50 Index Exchange Traded Fund,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,44280.0,7.263200e+04,3.216145e+09,False


In [5]:
stock_desc_df 


,Column,Sample value,Type,Addendum,Remarks
0,RowId,20170104_1301,string,NaN,Unique ID of price records
1,Date,2017-01-04 0:00:00,date,NaN,Trade date
2,SecuritiesCode,1301,Int64,NaN,Local securities code
3,Open,2734,float,NaN,first traded price on a day
4,High,2755,float,NaN,highest traded price on a day
5,Low,2730,float,NaN,lowest traded price on a day
6,Close,2742,float,NaN,last traded price on a day
7,Volume,31400,Int64,NaN,number of traded stocks on a day
8,AdjustmentFactor,1,float,NaN,to calculate theoretical price/volume when split/reverse-split happens (NOT including dividend/allotment of shares/)
9,SupervisionFlag,FALSE,boolean,NaN,Flag of Securities Under Supervision & Securities to Be Delisted\nhttps://www.jpx.co.jp/english/listing/market-alerts/supervision/00-archives/index.html )


In [6]:
stock_list_desc_df

,Column,Sample value,Type,Addendum,Remarks
0,SecuritiesCode,1301,Int64,NaN,Local Securities Code
1,EffectiveDate,20211230,date,NaN,the effective date
2,Name,"KYOKUYO CO.,LTD.",string,NaN,Name of security
3,Section/Products,First Section (Domestic),string,NaN,Section/Product
4,NewMarketSegment,Prime Market,string,NaN,New market segment effective from 2022-04-04 (as of 15:30 JST on Mar 11 2022)\nref. https://www.jpx.co.jp/english/equities/market-restructure/market-segments/index.html
5,33SectorCode,50,Int64,NaN,33 Sector Name\n\nref. https://www.jpx.co.jp/english/markets/indices/line-up/files/e_fac_13_sector.pdf
6,33SectorName,"Fishery, Agriculture and Forestry",string,NaN,33 Sector Name\n\nref. https://www.jpx.co.jp/english/markets/indices/line-up/files/e_fac_13_sector.pdf
7,17SectorCode,1,Int64,NaN,17 Sector Code\nref. https://www.jpx.co.jp/english/markets/indices/line-up/files/e_fac_13_sector.pdf
8,17SectorName,FOODS,string,NaN,17 Sector Name\nref. https://www.jpx.co.jp/english/markets/indices/line-up/files/e_fac_13_sector.pdf
9,NewIndexSeriesSizeCode,7,Int64,NaN,TOPIX New Index Series code\n\nref. https://www.jpx.co.jp/english/markets/indices/line-up/files/e_fac_12_size.pdf


In [7]:
stock_price_df = stock_price_df[~((stock_price_df['Open'].isna()) & (stock_price_df['High'].isna()) & (stock_price_df['Low'].isna()) & (stock_price_df['Close'].isna()))]

## Additional Features

### Metrics
BOP: Balance of Power = (Close price – Open price) / (High price – Low price) <br>
av: average = (Close price + High price + Low price) / 3 <br>
vwap: Volume-weighted average price = (Close price * Volume) / Volume <br>
HLr: High-Low-Range = High price – Low price <br>
OCr: Open-Close-Range = Close price – Open price <br>
OC: Open-Close = Close price * Open price <br>
relC: relative Change = (Close price – Open price)/Open Price <br>
HL: High-Low = High price * Low price <br>
logC: logarithmic scaled Close price <br>
logR: log(Close price) - log(Open price) <br>
OHLCstd: Open, High, Low, Close, standard deviated. <br>
OHLCskew: Open, High, Low, Close, skewed. <br>
OHLCkur: Open, High, Low, Close, kurtosis. <br>
Cpos: = (Close price – Low price) / (High price – Low price) -0.5 <br>
Opos: = (Open price – Low price) / (High price – Low price) -0.5 <br>
bsforce: = Cpos * Volume <br>
    
    
### Weekdays
Weekday = Day of the Week (1-5) <br>
Monday = if Monday == 1 <br>
Tuesday = if Tuesday == 1 <br>
Wednesday = if Wednesday == 1 <br>
Thursday = if Thursday == 1 <br>
Friday = if Fryday == 1 <br>
Date = Date

In [8]:
def FE(stock_price_df):
    stock_price_df['BOP'] = (stock_price_df['Open']-stock_price_df['Close'])/(stock_price_df['High']-stock_price_df['Low'])
    stock_price_df['Cpos'] = (stock_price_df['Close']-stock_price_df['Low'])/(stock_price_df['High']-stock_price_df['Low']) -0.5
    stock_price_df['bsforce'] = stock_price_df['Cpos'] * stock_price_df['Volume']
    stock_price_df['Opos'] = (stock_price_df['Open']-stock_price_df['Low'])/(stock_price_df['High']-stock_price_df['Low']) -0.5


    
    stock_price_df['av'] = (stock_price_df['Close']+stock_price_df['High']+stock_price_df['Low'])/3
    stock_price_df['vwap'] = (stock_price_df['Close'] * stock_price_df['Volume'])/ stock_price_df['Volume']
    stock_price_df['HLr'] = stock_price_df['High'] - stock_price_df['Low']
    stock_price_df['OCr'] = stock_price_df['Open'] - stock_price_df['Close']
    stock_price_df['relC'] = (stock_price_df['Close'] - stock_price_df['Open'])/stock_price_df['Open']
    stock_price_df['OC'] = stock_price_df['Open'] * stock_price_df['Close']
    stock_price_df['HL'] = stock_price_df['High'] * stock_price_df['Low']
    stock_price_df['logC'] = np.log(stock_price_df['Close']+1)
    stock_price_df['logR'] = np.log(stock_price_df['Close'])-np.log(stock_price_df['Open'])
    stock_price_df['OHLCstd'] = stock_price_df[['Open','High','Low','Close']].std(axis=1)
    stock_price_df['OHLCskew'] = stock_price_df[['Open','High','Low','Close']].skew(axis=1)
    stock_price_df['OHLCkur'] = stock_price_df[['Open','High','Low','Close']].kurtosis(axis=1)


    
    stock_price_df['weekday'] = stock_price_df['Date'].dt.weekday+1
    stock_price_df['Monday'] = np.where(stock_price_df['weekday']==1,1,0)
    stock_price_df['Tuesday'] = np.where(stock_price_df['weekday']==2,1,0)
    stock_price_df['Wednesday'] = np.where(stock_price_df['weekday']==3,1,0)
    stock_price_df['Thursday'] = np.where(stock_price_df['weekday']==4,1,0)
    stock_price_df['Friday'] = np.where(stock_price_df['weekday']==5,1,0)
    stock_price_df['Date'] = pd.to_datetime(stock_price_df['Date'])

    #rolling mean
    stock_price_df['SMA5'] = stock_price_df.Close.rolling(5).mean()
    stock_price_df['SMA20'] = stock_price_df.Close.rolling(20).mean()
    stock_price_df['SMA50'] = stock_price_df.Close.rolling(50).mean()
    stock_price_df['SMA200'] = stock_price_df.Close.rolling(200).mean()
    stock_price_df['SMA500'] = stock_price_df.Close.rolling(500).mean()

    #Exponentially Weighted average
    stock_price_df['EMA5'] = stock_price_df.Close.ewm(span=5, adjust=False).mean()
    stock_price_df['EMA20'] = stock_price_df.Close.ewm(span=20, adjust=False).mean()
    stock_price_df['EMA50'] = stock_price_df.Close.ewm(span=50, adjust=False).mean()
    stock_price_df['EMA200'] = stock_price_df.Close.ewm(span=200, adjust=False).mean()
    stock_price_df['EMA500'] = stock_price_df.Close.ewm(span=500, adjust=False).mean()

    

    return stock_price_df
stock_price_df = FE(stock_price_df)
stock_price_df = pd.merge(stock_price_df,stock_list, on='SecuritiesCode')

In [9]:
stock_price_df.head(10)

,RowId,Date,SecuritiesCode,Open,High,Low,Close_x,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,BOP,Cpos,bsforce,Opos,av,vwap,HLr,OCr,relC,OC,HL,logC,logR,OHLCstd,OHLCskew,OHLCkur,weekday,Monday,Tuesday,Wednesday,Thursday,Friday,SMA5,SMA20,SMA50,SMA200,SMA500,EMA5,EMA20,EMA50,EMA200,EMA500,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,NewIndexSeriesSize,TradeDate,Close_y,IssuedShares,MarketCapitalization,Universe0
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730,-0.320000,-0.020000,-628.000000,-0.340000,2742.333333,2742.0,25.0,-8.0,0.002926,7496628.0,7521150.0,7.916807,0.002922,11.026483,0.941530,0.008495,3,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,2742.000000,2742.000000,2742.000000,2742.000000,2742.000000,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,10928283.0,3.365911e+10,True
1,20170105_1301,2017-01-05,1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,NaN,False,0.002920,0.416667,-0.250000,-4475.000000,0.166667,2740.000000,2738.0,12.0,5.0,-0.001823,7510334.0,7513045.0,7.915348,-0.001824,5.315073,0.198134,-2.215052,4,0,0,0,1,0,1634.4,4667.40,3253.22,2227.1705,2036.1456,2469.865742,4128.913330,3611.159936,2591.513906,2244.800000,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,10928283.0,3.365911e+10,True
2,20170106_1301,2017-01-06,1301,2734.0,2744.0,2720.0,2740.0,19900,1.0,NaN,False,-0.001092,-0.250000,0.333333,6633.333333,0.083333,2734.666667,2740.0,24.0,-6.0,0.002195,7491160.0,7463680.0,7.916078,0.002192,10.503968,-1.164860,1.085094,5,0,0,0,0,1,1646.2,4653.20,3252.34,2236.6775,2038.7184,2478.198372,4120.312180,3608.506609,2593.691421,2246.059444,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,10928283.0,3.365911e+10,True
3,20170110_1301,2017-01-10,1301,2745.0,2754.0,2735.0,2748.0,24200,1.0,NaN,False,-0.005100,-0.157895,0.184211,4457.894737,0.026316,2745.666667,2748.0,19.0,-3.0,0.001093,7543260.0,7532190.0,7.918992,0.001092,7.937254,-0.703934,1.125220,2,0,1,0,0,0,1654.0,4516.45,3201.24,2231.0465,2037.7990,2467.890060,4024.784152,3538.924414,2575.661754,2240.805306,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,10928283.0,3.365911e+10,True
4,20170111_1301,2017-01-11,1301,2748.0,2752.0,2737.0,2745.0,9300,1.0,NaN,False,-0.003295,0.200000,0.033333,310.000000,0.233333,2744.666667,2745.0,15.0,3.0,-0.001092,7543260.0,7532224.0,7.917901,-0.001092,6.350853,-0.843252,0.933953,3,0,0,1,0,0,1662.8,4516.15,3178.76,2195.1835,2024.0430,2465.875000,4019.587059,3531.507288,2555.135526,2224.430970,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,10928283.0,3.365911e+10,True
5,20170112_1301,2017-01-12,1301,2745.0,2747.0,2703.0,2731.0,28700,1.0,NaN,False,-0.006613,0.318182,0.136364,3913.636364,0.454545,2727.000000,2731.0,44.0,14.0,-0.005100,7496595.0,7425141.0,7.912789,-0.005113,20.289570,-1.354079,1.265400,4,0,0,0,1,0,1654.0,4516.35,3175.50,2210.6435,2031.7356,2460.148352,4018.875270,3531.482430,2564.757461,2233.126189,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,10928283.0,3.365911e+10,True
6,20170113_1301,2017-01-13,1301,2707.0,2730.0,2707.0,2722.0,19400,1.0,NaN,False,-0.006657,-0.652174,0.152174,2952.173913,-0.500000,2719.666667,2722.0,23.0,-15.0,0.005541,7368454.0,7390110.0,7.909489,0.005526,11.445523,0.405505,-3.706427,5,0,0,0,0,1,1640.0,4398.10,3140.56,2188.8645,2013.0798,2425.477972,3935.484181,3475.322279,2536.435570,2211.649989,2

In [10]:
df = stock_price_df.drop(columns=['17SectorCode','NewIndexSeriesSizeCode', '33SectorCode', 'ExpectedDividend', 'RowId', 'Close_y'])

In [11]:
df.rename(columns={"Close_x": "Close"}, inplace=True)

In [12]:
df.head()

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,SupervisionFlag,Target,BOP,Cpos,bsforce,Opos,av,vwap,HLr,OCr,relC,OC,HL,logC,logR,OHLCstd,OHLCskew,OHLCkur,weekday,Monday,Tuesday,Wednesday,Thursday,Friday,SMA5,SMA20,SMA50,SMA200,SMA500,EMA5,EMA20,EMA50,EMA200,EMA500,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorName,17SectorName,NewIndexSeriesSize,TradeDate,IssuedShares,MarketCapitalization,Universe0
0,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,False,0.000730,-0.320000,-0.020000,-628.000000,-0.340000,2742.333333,2742.0,25.0,-8.0,0.002926,7496628.0,7521150.0,7.916807,0.002922,11.026483,0.941530,0.008495,3,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,2742.000000,2742.000000,2742.000000,2742.000000,2742.000000,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,"Fishery, Agriculture and Forestry",FOODS,TOPIX Small 2,20211230.0,10928283.0,3.365911e+10,True
1,2017-01-05,1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,False,0.002920,0.416667,-0.250000,-4475.000000,0.166667,2740.000000,2738.0,12.0,5.0,-0.001823,7510334.0,7513045.0,7.915348,-0.001824,5.315073,0.198134,-2.215052,4,0,0,0,1,0,1634.4,4667.40,3253.22,2227.1705,2036.1456,2469.865742,4128.913330,3611.159936,2591.513906,2244.800000,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,"Fishery, Agriculture and Forestry",FOODS,TOPIX Small 2,20211230.0,10928283.0,3.365911e+10,True
2,2017-01-06,1301,2734.0,2744.0,2720.0,2740.0,19900,1.0,False,-0.001092,-0.250000,0.333333,6633.333333,0.083333,2734.666667,2740.0,24.0,-6.0,0.002195,7491160.0,7463680.0,7.916078,0.002192,10.503968,-1.164860,1.085094,5,0,0,0,0,1,1646.2,4653.20,3252.34,2236.6775,2038.7184,2478.198372,4120.312180,3608.506609,2593.691421,2246.059444,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,"Fishery, Agriculture and Forestry",FOODS,TOPIX Small 2,20211230.0,10928283.0,3.365911e+10,True
3,2017-01-10,1301,2745.0,2754.0,2735.0,2748.0,24200,1.0,False,-0.005100,-0.157895,0.184211,4457.894737,0.026316,2745.666667,2748.0,19.0,-3.0,0.001093,7543260.0,7532190.0,7.918992,0.001092,7.937254,-0.703934,1.125220,2,0,1,0,0,0,1654.0,4516.45,3201.24,2231.0465,2037.7990,2467.890060,4024.784152,3538.924414,2575.661754,2240.805306,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,"Fishery, Agriculture and Forestry",FOODS,TOPIX Small 2,20211230.0,10928283.0,3.365911e+10,True
4,2017-01-11,1301,2748.0,2752.0,2737.0,2745.0,9300,1.0,False,-0.003295,0.200000,0.033333,310.000000,0.233333,2744.666667,2745.0,15.0,3.0,-0.001092,7543260.0,7532224.0,7.917901,-0.001092,6.350853,-0.843252,0.933953,3,0,0,1,0,0,1662.8,4516.15,3178.76,2195.1835,2024.0430,2465.875000,4019.587059,3531.507288,2555.135526,2224.430970,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,"Fishery, Agriculture and Forestry",FOODS,TOPIX Small 2,20211230.0,10928283.0,3.365911e+10,True


In [13]:
df.shape

(2324923, 53)

### Additional features with ta library

In [14]:
import ta as ta
from ta import add_all_ta_features
from ta.utils import dropna

In [15]:
df = ta.add_all_ta_features(df, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=False)

In [16]:
df.sample(10)

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,SupervisionFlag,Target,BOP,Cpos,bsforce,Opos,av,vwap,HLr,OCr,relC,OC,HL,logC,logR,OHLCstd,OHLCskew,OHLCkur,weekday,Monday,Tuesday,Wednesday,Thursday,Friday,SMA5,SMA20,SMA50,SMA200,SMA500,EMA5,EMA20,EMA50,EMA200,EMA500,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorName,17SectorName,NewIndexSeriesSize,TradeDate,IssuedShares,MarketCapitalization,Universe0,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,volume_sma_em,volume_vpt,volume_vwap,volume_mfi,volume_nvi,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_atr,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,trend_sma_slow,trend_ema_fast,trend_ema_slow,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_stc,trend_adx,trend_adx_pos,trend_adx_neg,trend_cci,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
1802503,2017-11-15,8278,2356.0,2409.0,2313.0,2322.0,1049200,1.0,False,0.004659,0.354167,-0.406250,-426237.500000,-0.052083,2348.000000,2322.0,96.0,34.0,-0.014431,5470632.0,5572017.0,7.750615,-0.014536,43.474130,1.079034,0.131080,3,0,0,1,0,0,2592.8,2689.400,2319.360,2834.4920,2921.6534,2476.019571,2630.971703,2514.929459,2700.084619,2844.861972,20211230,"FUJI CO.,LTD.",First Section (Domestic),Prime Market,Retail Trade,RETAIL TRADE,TOPIX Small 2,20211230.0,38291560.0,7.478342e+10,True,-1.299683e+09,52751742500,-0.660585,-1.613155e+07,-2.287457e+05,-2.578908e+06,-21321.687544,2499.068511,0.000000,8.347367e-94,2752.85,3332.311224,2173.388776,42.099005,0.128232,0.0,0.0,2500.466667,2556.766667,2444.166667,4.503159,-1.084962,0.0,1.0,2313.0,3150.0,2731.5,30.404853,0.010753,71.595407,17.964198,-153.844003,-100.069439,-53.774565,2542.250000,2816.923077,2538.582147,2692.426151,0.449074,1.453704,-1.004630,-0.473631,25.741862,184.15,-128.884790,-40.598402,-88.286388,2496.5,2774.0,2635.25,2774.0,0.006092,42.189051,11.128350,49.432482,-102.996963,2832.00,2784.5,40.0,100.0,-60.0,NaN,2573.960001,0.0,0.0,16.891403,0.000000,0.000000,0.000000,-49.238806,14.536517,1.173403,0.743704,-98.826597,-428.908824,-22.728785,-5.713954,-3.655701,-2.058254,51.319253,43.452942,7.866311,NaN,-1.443124,-1.453638,-15.317287
1947320,2020-09-30,8877,1375.0,1382.0,1341.0,1354.0,48200,1.0,False,0.002954,0.512195,-0.182927,-8817.073171,0.329268,1359.000000,1354.0,41.0,21.0,-0.015273,1861750.0,1853262.0,7.211557,-0.015391,18.885621,-0.276134,-3.231662,3,0,0,1,0,0,1242.2,1587.925,1368.036,1792.5925,2202.5864,1463.214843,1479.413261,1473.636178,1777.366785,2146.157271,20211230,ESLEAD CORPORATION,First Section (Domestic),Prime Market,Real Estate,REAL ESTATE,TOPIX Small 2,20211230.0,15465600.0,2.605954e+10,True,-6.660063e+09,56157386200,0.081077,-7.787517e+04,-1.403527e+06,2.769778e+05,-749.000492,1346.821484,66.475469,2.741712e-81,1335.50,1390.594464,1280.405536,8.250762,0.667893,0.0,0.0,1353.466667,1383.866667,1323.066667,4.492168,0.508772,0.0,0.0,1292.0,1399.0,1345.5,8.011981,0.579439,31.039637,1.441241,20.105985,16.134461,3.971524,1351.333333,1323.807692,1352.965958,1332.859973,1.054054,0.793612,0.260442,0.187107,24.598868,-7.50,55.529764,39.155000,16.374764,1355.

In [17]:
df.isnull().sum() * 100 / len(df)

Date                          0.000000
SecuritiesCode                0.000000
Open                          0.000000
High                          0.000000
Low                           0.000000
Close                         0.000000
Volume                        0.000000
AdjustmentFactor              0.000000
SupervisionFlag               0.000000
Target                        0.000000
BOP                           0.423154
Cpos                          0.423154
bsforce                       0.423154
Opos                          0.423154
av                            0.000000
vwap                          0.000000
HLr                           0.000000
OCr                           0.000000
relC                          0.000000
OC                            0.000000
HL                            0.000000
logC                          0.000000
logR                          0.000000
OHLCstd                       0.000000
OHLCskew                      0.000000
OHLCkur                  

In [18]:
df = df.drop(columns=['momentum_kama','trend_psar_up', 'trend_psar_down'])

In [19]:
df.head()

,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,SupervisionFlag,Target,BOP,Cpos,bsforce,Opos,av,vwap,HLr,OCr,relC,OC,HL,logC,logR,OHLCstd,OHLCskew,OHLCkur,weekday,Monday,Tuesday,Wednesday,Thursday,Friday,SMA5,SMA20,SMA50,SMA200,SMA500,EMA5,EMA20,EMA50,EMA200,EMA500,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorName,17SectorName,NewIndexSeriesSize,TradeDate,IssuedShares,MarketCapitalization,Universe0,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,volume_sma_em,volume_vpt,volume_vwap,volume_mfi,volume_nvi,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_atr,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,trend_sma_slow,trend_ema_fast,trend_ema_slow,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_stc,trend_adx,trend_adx_pos,trend_adx_neg,trend_cci,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up_indicator,trend_psar_down_indicator,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,others_dr,others_dlr,others_cr
0,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,False,0.000730,-0.320000,-0.020000,-628.000000,-0.340000,2742.333333,2742.0,25.0,-8.0,0.002926,7496628.0,7521150.0,7.916807,0.002922,11.026483,0.941530,0.008495,3,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,2742.000000,2742.000000,2742.000000,2742.000000,2742.000000,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,"Fishery, Agriculture and Forestry",FOODS,TOPIX Small 2,20211230.0,10928283.0,3.365911e+10,True,-1256.000000,31400,NaN,NaN,NaN,NaN,3566.864417,NaN,NaN,1000.000000,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,2767.333333,2717.333333,NaN,0.493333,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2742.5,NaN,0.0,0.0,0.0,NaN,2598.913706,2612.040202,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.704519,NaN,0.000000
1,2017-01-05,1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,False,0.002920,0.416667,-0.250000,-4475.000000,0.166667,2740.000000,2738.0,12.0,5.0,-0.001823,7510334.0,7513045.0,7.915348,-0.001824,5.315073,0.198134,-2.215052,4,0,0,0,1,0,1634.4,4667.40,3253.22,2227.1705,2036.1456,2469.865742,4128.913330,3611.159936,2591.513906,2244.800000,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,"Fishery, Agriculture and Forestry",FOODS,TOPIX Small 2,20211230.0,10928283.0,3.365911e+10,True,-10206.000000,13500,NaN,NaN,-1.005587e+05,NaN,1765.106557,NaN,NaN,998.541211,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,2759.666667,2722.666667,NaN,0.414414,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2742.5,NaN,0.0,0.0,0.0,NaN,2598.913706,2612.040202,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.145879,-0.145985,-0.145879
2,2017-01-06,1301,2734.0,2744.0,2720.0,2740.0,19900,1.0,False,-0.001092,-0.250000,0.333333,6633.333333,0.083333,2734.666667,2740.0,24.0,-6.0,0.002195,7491160.0,7463680.0,7.916078,0.002192,10.503968,-1.164860,1.085094,5,0,0,0,0,1,1646.2,4653.20,3252.34,2236.6775,2038.7184,2478.198372,4120.312180,3608.506609,2593.691421,2246.059444,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,"Fishery, Agriculture and Forestry",FOODS,TOPIX Small 2,20211230.0,10928283.0,3.365911e+10,True,3060.66666

In [20]:
df.shape

(2324923, 136)

In [21]:
df.shape

(2324923, 136)

In [22]:
df.isnull().sum() * 100 / len(df)

Date                         0.000000
SecuritiesCode               0.000000
Open                         0.000000
High                         0.000000
Low                          0.000000
Close                        0.000000
Volume                       0.000000
AdjustmentFactor             0.000000
SupervisionFlag              0.000000
Target                       0.000000
BOP                          0.423154
Cpos                         0.423154
bsforce                      0.423154
Opos                         0.423154
av                           0.000000
vwap                         0.000000
HLr                          0.000000
OCr                          0.000000
relC                         0.000000
OC                           0.000000
HL                           0.000000
logC                         0.000000
logR                         0.000000
OHLCstd                      0.000000
OHLCskew                     0.000000
OHLCkur                      0.000000
weekday     

In [23]:
df.fillna(method='ffill', inplace=True)

In [24]:
df.isnull().sum() * 100 / len(df)

Date                         0.000000
SecuritiesCode               0.000000
Open                         0.000000
High                         0.000000
Low                          0.000000
Close                        0.000000
Volume                       0.000000
AdjustmentFactor             0.000000
SupervisionFlag              0.000000
Target                       0.000000
BOP                          0.000000
Cpos                         0.000000
bsforce                      0.000000
Opos                         0.000000
av                           0.000000
vwap                         0.000000
HLr                          0.000000
OCr                          0.000000
relC                         0.000000
OC                           0.000000
HL                           0.000000
logC                         0.000000
logR                         0.000000
OHLCstd                      0.000000
OHLCskew                     0.000000
OHLCkur                      0.000000
weekday     

In [25]:
df.fillna(method='backfill', inplace=True)

In [26]:
df.isnull().sum() * 100 / len(df)

Date                         0.0
SecuritiesCode               0.0
Open                         0.0
High                         0.0
Low                          0.0
Close                        0.0
Volume                       0.0
AdjustmentFactor             0.0
SupervisionFlag              0.0
Target                       0.0
BOP                          0.0
Cpos                         0.0
bsforce                      0.0
Opos                         0.0
av                           0.0
vwap                         0.0
HLr                          0.0
OCr                          0.0
relC                         0.0
OC                           0.0
HL                           0.0
logC                         0.0
logR                         0.0
OHLCstd                      0.0
OHLCskew                     0.0
OHLCkur                      0.0
weekday                      0.0
Monday                       0.0
Tuesday                      0.0
Wednesday                    0.0
Thursday  

In [27]:
df.corr()

,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,SupervisionFlag,Target,BOP,Cpos,bsforce,Opos,av,vwap,HLr,OCr,relC,OC,HL,logC,logR,OHLCstd,OHLCskew,OHLCkur,weekday,Monday,Tuesday,Wednesday,Thursday,Friday,SMA5,SMA20,SMA50,SMA200,SMA500,EMA5,EMA20,EMA50,EMA200,EMA500,EffectiveDate,TradeDate,IssuedShares,MarketCapitalization,Universe0,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,volume_sma_em,volume_vpt,volume_vwap,volume_mfi,volume_nvi,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_atr,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,trend_sma_slow,trend_ema_fast,trend_ema_slow,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_stc,trend_adx,trend_adx_pos,trend_adx_neg,trend_cci,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up_indicator,trend_psar_down_indicator,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,others_dr,others_dlr,others_cr
SecuritiesCode,1.000000e+00,0.016788,0.016348,0.017235,0.016798,0.040899,1.726997e-03,-0.000978,-3.351732e-03,-0.000594,-0.001088,-0.001178,-0.001983,0.016791,0.016798,-0.014810,-0.000316,0.000013,0.032826,0.032823,-0.018138,0.000910,-0.014717,0.002139,0.001576,0.000058,-0.000083,0.000014,0.000022,0.000028,1.518002e-05,0.036872,0.000509,-0.041766,-0.183562,0.003348,0.034993,0.018025,-0.027815,-0.131093,-0.065101,NaN,0.027762,6.020079e-02,0.066267,NaN,-0.710887,0.927066,0.000594,-0.001759,-4.306765e-04,-0.001634,-0.000088,0.016769,-0.011190,-0.105699,0.016936,0.014675,0.019000,-0.018209,-0.005086,-0.002856,0.001967,0.016856,0.015988,0.017762,-0.074700,1.073821e-03,-0.001194,0.005146,0.019075,0.014114,0.016509,-0.027343,-0.001270,-0.012686,-0.027342,-0.000531,-0.000570,1.499513e-08,0.016878,0.016978,0.016906,0.017032,-0.019432,-0.006233,-0.006865,-0.000241,0.000880,0.000080,0.000494,0.000506,-0.000005,0.016681,0.016411,0.016573,0.015930,-0.003565,-0.011343,-0.001921,0.006448,-0.004671,0.016700,0.016057,-0.005833,0.005548,-0.006501,0.000531,0.000515,-0.004395,-0.001419,-0.001567,-0.001636,-0.004586,0.000505,-0.001687,-0.001820,-0.001687,-0.000443,0.000063,0.000136,0.000146,-2.553094e-05,0.007099,0.008402,-0.000011,0.000013,-0.000041,0.016798
Open,1.678794e-02,1.000000,0.999851,0.999854,0.999730,-0.036524,-6.917870e-03,-0.004112,-3.919391e-03,-0.002756,0.000718,0.001492,-0.003807,0.999870,0.999730,0.735456,0.019476,-0.002797,0.827294,0.827282,0.692490,-0.002642,0.719691,0.003059,-0.006019,-0.000387,0.000472,-0.000156,0.000302,-0.000614,1.049898e-05,0.469875,0.259102,0.174033,0.094937,0.078379,0.746876,0.432588,0.288581,0.157973,0.111318,NaN,-0.028370,-8.211040e-05,0.373909,NaN,-0.036477,0.034423,0.036563,0.010035,9.644037e-03,0.036913,0.002740,0.992426,0.036660,-0.001685,0.990068,0.969766,0.978727,0.002764,0.043511,0.015336,-0.012949,0.995335,0.995188,0.995199,-0.017795,9.066570e-05,0.033171,-0.017921,0.990067,0.980702,0.992022,0.001135,0.055468,0.785786,-0.057689,0.170565,0.178682,9.774462e-03,0.994306,0.986980,0.995999,0.990606,0.039212,-0.059174,0.050996,0.104847,0.051568,-0.034129,0.078815,0.079474,0.003458,0.996580,0.989552,0.994686,0.978711,-0.020524,-0.000582,0.102863,-0.061480,0.044028,0.955297,0.939223,0.055924,-0.043690,0.056910,0.000263,0.004279,0.095000,-0.008613,-0.005670,-0.004428,0.113042,0.044806,0.044343,0.051106,0.044343,0.141730,0.050559

In [28]:
df.shape

(2324923, 136)

## Notes
### Potential Features

- Earthquakes?